In [1]:
import websockets
from wxasync import WxAsyncApp
import os
import asyncio
from bfxapi import Client
from bfxapi.utils.CustomLogger import CustomLogger
from exchange.BFX import wsbfx
from exchange.wx_gui import WalletFrame, WalletEvent
from exchange.BFX.tickerdata import TickerData

In [2]:
api_key = os.getenv('BFX_KEY')
api_secret = os.getenv('BFX_SECRET')
log = CustomLogger('Jupyter', 'INFO')
mySubscribed = {}
myWallet = dict()
tickerDict = dict()
base_currency = 'USD'
tickerDataFields = ['bid', 'bid_size', 'ask', 'ask_size', 'daily_change',
                    'daily_change_freq', 'last_price', 'volume', 'high', 'low']

In [3]:
def bfxMonitor(bfx):
    @bfx.ws.on('wallet_snapshot')
    async def log_snapshot(wallets):
        for wallet in wallets:
            if wallet.type == 'exchange' and wallet.currency not in ['SPK', 'USD']:  # No small balances
                myWallet[wallet.currency] = wallet.balance
                sym = 't' + wallet.currency + base_currency
                await bfx.ws.subscribe('ticker', sym)
            if wallet.type == ' exchange' and wallet.currency =='USD':
                myWallet[wallet.currency] = wallet.balance

    @bfx.ws.on('wallet_update')
    async def log_update(wallet):
        if wallet.currency not in myWallet.keys():
            myWallet[wallet.currency] = wallet.balance
            sym = 't' + wallet.currency + base_currency
            await bfx.ws.subscribe('ticker', sym)
#         log.info("Wallet updates: {}".format(wallet))

    @bfx.ws.on('subscribed')
    def show_channel(sub):
        coin = sub.symbol[1:4]
        mySubscribed[sub.chan_id] = coin
        log.info(f"{coin} is subscribed  - id:- {sub.chan_id}")
        tickerDict[coin] = TickerData(symbol=sub.symbol, channel_name=sub.channel_name,
                                      chan_id=sub.chan_id, sub_id=sub.sub_id,
                                      balance=myWallet[coin])
        log.info(f"tickerDict: {tickerDict[coin]}")
        
    @bfx.ws.on('unsubscribed')
    async def log_unsubscribe(sub):
        log.info(f"unsubscribed: {sub}")
        del(tickerDict[sub.symbol[1:4]])
        
        
    @bfx.ws.on('error')
    async def log_error(msg):
        log.error("Error: {}".format(msg))

    @bfx.ws.on('balance_update')
    async def log_balance(data):
#         log.info('Balance update: {}'.format(data))
        pass
    
    @bfx.ws.on('ticker_update')
    async def log_ticker(tkdata):
        symbol, tkupdate = tkdata
        data = dict(zip(tickerDataFields, tkupdate))
        tickerDict[symbol].update(**data)
#         log.info(f"MyWallet: {symbol} - ${tickerDict[symbol].wallet:2f}")


In [4]:
async def main_async():
    loop = asyncio.get_running_loop()
    bfx = Client(API_KEY=api_key, API_SECRET=api_secret, loop=loop)

    tasks = [
        asyncio.ensure_future(bfx.ws.get_task_executable()),
    ]
    bfxMonitor(bfx)
    await asyncio.wait(tasks)

In [5]:
await main_async()

[BfxWebsocket] [INFO] Wesocket connected to wss://api-pub.bitfinex.com/ws/2
[BfxWebsocket] [INFO] {'event': 'info', 'version': 2, 'serverId': '0327bbca-4721-45d4-b6f7-c7115e1fca1c', 'platform': {'status': 1}}
[BfxWebsocket] [INFO] Websocket opened.
[BfxWebsocket] [INFO] Authentication successful.
[BfxWebsocket] [INFO] Position snapshot: [0, 'ps', []]
[BfxSubscriptionManager] [INFO] Subscribing to channel ticker
[BfxSubscriptionManager] [INFO] Subscribing to channel ticker
[BfxSubscriptionManager] [INFO] Subscribing to channel ticker
[BfxSubscriptionManager] [INFO] Subscribing to channel ticker
[BfxWebsocket] [INFO] Funding offer snapshot: [0, 'fos', []]
[BfxWebsocket] [INFO] Funding credit snapshot: [0, 'fcs', []]
[BfxWebsocket] [INFO] Funding loan snapshot: [0, 'fls', []]
[BfxWebsocket] [INFO] Wallet update: Wallet <'exchange_SPK' balance='1e-08' unsettled='0'>
[BfxSubscriptionManager] [INFO] Subscribing to channel ticker
[BfxWebsocket] [INFO] Wallet update: Wallet <'exchange_ETC' bal

CancelledError: 